# NRPy+ Tutorial

## Introduction & Motivation

NRPy+ ("Python-based code generation for numerical relativity... and beyond!") is a Python-based code, in which user-specified expressions in indexed notation are output as highly optimized C code. It makes extensive use of [SymPy](http://www.sympy.org) for algebraic manipulation.

As NRPy+ was originally designed for generating C code at the heart of finite-difference-based numerical relativity codes, it contains a number of modules for solving PDEs that are useful for a large class of [hyperbolic systems of PDEs](https://en.wikipedia.org/wiki/Hyperbolic_partial_differential_equation), like the [scalar wave equation](https://en.wikipedia.org/wiki/Wave_equation).

NRPy+ also contains functionality for dealing with indexed expressions. After initializing the basic infrastructure

In [3]:
# The NRPy_param_funcs module sets up global structures that manage free parameters within NRPy+
import NRPy_param_funcs as par 
# The indexedexp module defines various functions for defining and managing indexed quantities like tensors and pseudotensors
import indexedexp as ixp
# The grid module defines various parameters related to a numerical grid or the dimensionality of indexed expressions
# For example, it declares the parameter DIM, which specifies the dimensionality of the indexed expression
import grid as gri

Indexed expressions of rank 1 are stored as Python lists; rank-2 indexed expressions are stored as lists of lists; etc. In addition, the covariance ("down" indices) or contravariance ("up" indices) of indexed expressions are specified in the variable name with D's or U's, respectively. For example
+ the contravariant rank-1 tensor $\beta^i$ is declared as $\text{betaU[i]}$ in NRPy+, so that e.g., $\text{betaU[0]}$ is stored as the SymPy variable named "betaU0"
+ the covariant rank-2 tensor $g_{ij}$ is declared as $\text{gDD[i][j]}$ in NRPy+, so that e.g., $\text{gDD[0][2]}$ is stored as the SymPy variable named "gDD02" and
+ the rank-2 tensor $T^{\mu}{}_{\nu}$ is declared as $\text{TUD[m][n]}$ in NRPy+ (index names are of course arbitrary).

*Caveat*: Note that it is currently up to the user to determine whether the combination of indexed expressions makes sense; NRPy+ does not track whether up and down indices are written consistently.

NRPy+ supports symmetries in indexed expressions, so that if $h_{ij} = h_{ji}$, then declaring $\text{hDD[i][j]}$ to be symmetric in NRPy+ will result in both $\text{hDD[0][2]}$ and $\text{hDD[2][0]}$ mapping to the *single* SymPy variable $hDD02$.

To see how this works in NRPy+, let's define in NRPy+ a symmetric, rank-2 tensor $h_{ij}$ in three dimensions, and then compute the contraction, which should be given by $$con = h^{ij}h_{ij} = h_{00} h^{00} + h_{11} h^{11} + h_{22} h^{22} + 2 (h_{01} h^{01} + h_{02} h^{02} + h_{12} h^{12})$$.

In [16]:
# Set the dimension to 3
par.set_parval_from_str("DIM",3)

# Get the dimension we just set.
DIM = par.parval_from_str("DIM")

# Declare h_{ij}=hUU[i][j] and h^{ij}=hDD[i][j]
hUU = ixp.declarerank2("hUU","sym12")
hDD = ixp.declarerank2("hDD","sym12")

# Perform sum h^{ij} h_{ij}, initializing contraction result to zero
con = 0
for i in range(DIM):
    for j in range(DIM):
        con += hUU[i][j]*hDD[i][j]

# Print result
print(con)

hDD00*hUU00 + 2*hDD01*hUU01 + 2*hDD02*hUU02 + hDD11*hUU11 + 2*hDD12*hUU12 + hDD22*hUU22


## Part 1: Generating C code to solve the scalar wave equation, in Cartesian coordinates

### Problem Statement

We wish to numerically solve the scalar wave equation in Cartesian coordinates:
$$\partial_t^2 u = c^2 \nabla^2 u \text{,}$$
where $u$ is a function of time and space: $u = u(t,x,y,...)$ (spatial dimension as-yet unspecified) and $c$ is the wave speed,

subject to some initial condition
$$u(0,x,y,...) = f(x,y,...)$$

and suitable, approximate spatial boundary conditions.

Define 
$$v(t,x,y,...) = \partial_t u(t,x,y,...)$$

In this way, the second-order PDE is reduced to a set of two coupled first-order PDEs

\begin{align}
\partial_t u &= v \\
\partial_t v &= c^2 \nabla^2 u.
\end{align}


### Numerical Solution to the Wave Equation 

We will find that breaking the second-order time derivative into two first-order, coupled equations in this way is quite convenient numerically, as it enables us to step forward in time using the Method of Lines. This method enables us to handle timestepping the same way we would handle an ODE integration, allowing us to use [Runge-Kutta methods](https://en.wikipedia.org/wiki/Runge%E2%80%93Kutta_methods) for timestepping. 

As for the $\nabla^2 u$ term, spatial derivatives are handled in NRPy+ via [finite differencing](https://en.wikipedia.org/wiki/Finite_difference).

We will sample the solution $\{u,v\}$ at discrete, uniformly-sampled points in space and time For simplicity, let's assume that we consider the wave equation in one spatial dimension. Then the solution at any sampled point in space and time is given by
$$u^n_i = u(t_n,x_i) = u(t_0 + n \Delta t, x_0 + i \Delta x),$$
where $\Delta t$ and $\Delta x$ represent the temporal and spatial resolution, respectively. $v^n_i$ is sampled at the same points in space and time.

#### Spatial Derivatives: the $\nabla^2$ Operator

To minimize complication, we will restrict ourselves to solving the wave equation in one spatial dimension, so
$$\nabla^2 u = \partial_x^2 u.$$
Extension of this operator to higher spatial dimensions is straightforward, particularly when using NRPy+.

We wish to approximate this derivative using [finite difference](https://en.wikipedia.org/wiki/Finite_difference). (FD) techniques. FD techniques are usually equivalent to first finding the unique $N$th degree polynomial that passes through $N+1$ sampled points of our function ($u$) in the neighborhood of where we wish to find the derivative. Then, provided $u$ is smooth and properly-sampled the derivative of the polynomial is an approximate derivative of $u$. The approximation error generally decreases as the polynomial degree or sampling density increases.

Finite difference derivatives are written in the form
$$\left[\partial_x u\right]_i = \sum_{j} \left(u_j\right) a_j,$$
where the $a_j$s are known as *finite difference coefficients*. For a given finite difference representation (corresponding to a fixed polynomial degree), the set of $a_j$s are unique.

For example, the second-order derivative $\partial_x^2$ accurate to fourth-order in uniform grid spacing $\Delta x$ (from fitting the unique 4th-degree polynomial to 5 sample points of $u$) is given by
\begin{equation}
\left[\partial_x^2 u(t,x)\right]_j = \frac{1}{(\Delta x)^2}
\left(
-\frac{1}{12} \left(u_{j+2} + u_{j-2}\right) 
+ \frac{4}{3}  \left(u_{j+1} + u_{j-1}\right)
- \frac{5}{2} u_j \right)
+ \mathcal{O}\left((\Delta x)^4\right).
\end{equation}

As another example, the same derivative accurate to second-order in uniform grid spacing $\Delta x$ (from fitting the unique 4th-degree polynomial to 3 sample points of $u$) is given by
\begin{equation}
\left[\partial_x^2 u(t,x)\right]_j = \frac{1}{(\Delta x)^2}
\left(u_{j+1} - 2 u_j + u_{j-1}\right)
+ \mathcal{O}\left((\Delta x)^2\right).
\end{equation}

For more details on how finite difference coefficients are computed (it's not magic!), as well as its implementation in NRPy+, see the Appendix at the bottom of this notebook.

#### Temporal Derivatives: Numerical Timestepping




In [1]:
import NRPy_param_funcs as par
import indexedexp as ixp
import grid as gri
import finite_difference as fin
from outputC import *

In [2]:
# The name of this module ("scalarwave") is given by __name__:
thismodule = __name__

par.set_parval_from_str("DIM",1)

# Step 0: Get the spatial dimension, as it was set in 
#         the grid module.
DIM = par.parval_from_str("DIM")

# Step 1: Register gridfunctions that are needed as input.
uu, vv = gri.register_gridfunctions("EVOL",["uu","vv"])

# Step 2: Declare the rank-2 indexed expression \partial_{ij} u,
#         which is symmetric about interchange of indices i and j
#         Derivative variables like these must have an underscore
#         in them, so the finite difference module can parse the
#         variable name properly.
uu_dDD = ixp.declarerank2("uu_dDD","sym12")

# Step 3: Define the C parameter wavespeed. The `wavespeed`
#         variable is a proper SymPy variable, so it can be
#         used in below expressions. In the C code, it acts
#         just like a usual parameter, whose value is 
#         specified in the parameter file.
wavespeed = par.Cparameters("REAL",thismodule,"wavespeed")

# Step 4: Define right-hand sides for the evolution.
uu_rhs = vv
vv_rhs = 0
for i in range(DIM):
    vv_rhs += wavespeed*wavespeed*uu_dDD[i][i]

vv_rhs = sp.simplify(vv_rhs)
    
# Step 5: Generate C code for scalarwave evolution equations,
#         print output to the screen (standard out, or stdout).
fin.FD_outputC("stdout",
               [lhrh(lhs=gri.gfaccess("out_gfs","UU_rhs"),rhs=uu_rhs),
                lhrh(lhs=gri.gfaccess("out_gfs","VV_rhs"),rhs=vv_rhs)])


/*
 *  Original SymPy expressions:
 *  "[const double uu_dDD00 = invdx0**2*(-5*uu/2 + 4*uu_i0m1/3 - uu_i0m2/12 + 4*uu_i0p1/3 - uu_i0p2/12),
 *    out_gfs[IDX2(UU_rhs, i0)] = vv,
 *    out_gfs[IDX2(VV_rhs, i0)] = uu_dDD00*wavespeed**2]"
 */
{
   const double uu_i0m2 = in_gfs[IDX2(UUGF, i0-2)];
   const double uu_i0m1 = in_gfs[IDX2(UUGF, i0-1)];
   const double uu = in_gfs[IDX2(UUGF, i0)];
   const double uu_i0p1 = in_gfs[IDX2(UUGF, i0+1)];
   const double uu_i0p2 = in_gfs[IDX2(UUGF, i0+2)];
   const double vv = in_gfs[IDX2(VVGF, i0)];
   const double uu_dDD00 = pow(invdx0, 2)*(-(5.0 / 2.0)*uu + ((4.0 / 3.0))*uu_i0m1 - (1.0 / 12.0)*uu_i0m2 + ((4.0 / 3.0))*uu_i0p1 - (1.0 / 12.0)*uu_i0p2);;
   out_gfs[IDX2(UU_rhs, i0)] = vv;;
   out_gfs[IDX2(VV_rhs, i0)] = uu_dDD00*pow(wavespeed, 2);;
}



### Appendix: How NRPy+ Computes Finite Difference Coefficients

Recall that finite difference derivatives are written in the form
$$\left[\partial_x u\right]_i = \sum_{j} \left(u_j\right) a_j,$$
where the $a_j$s are known as *finite difference coefficients*. For a given finite difference representation, the set of $a_j$s are unique.

There are many ways to compute finite difference coefficients $a_j$, though perhaps the most popular involves Taylor series expansions about sampled points near the point where we wish to evaluate the derivative. 

As an illustration, let's first derive for uniform grids the centered, first-order, finite-difference coefficients accurate to fourth-order in $\Delta x$. A fourth-order-accurate centered finite-difference derivative is equivalent to the derivative of the unique polynomial fit at sampled points $\left\{j-2,j-1,j,j+1,j+2\right\}$. 

The Taylor series expansion of a function $f(x)$ about a point $x_0$ is given by

$$f(x) = \sum_{n=0}^\infty \frac{f^{(n)}(x_0)}{n!} (x-x_0)^n,$$

where $f^{(n)}(x_0)$ is the $n$th derivative of $f$ evaluated at point $x_0$. Based on this, we can immediately write the Taylor expansion of $f$ at a point $x=x_0+j\Delta x$. In this case,

\begin{align}
f(x_0+j\Delta x) &= \sum_{n=0}^\infty \frac{f^{(n)}(x_0)}{n!} (j\Delta x)^n,\text{ or equivalently:} \\
f_j &= \sum_{n=0}^\infty \frac{f^{(n)}_j}{n!} (j\Delta x)^n.
\end{align}

Thus Taylor expanding the solution $u$ about the sampled points $\left\{j-2,j-1,j,j+1,j+2\right\}$ to $n=5$ yields the following:

\begin{align}
u_{j-2} &= u_j - (2 \Delta x) u'_j + \frac{(2 \Delta x)^2}{2} u''_j - \frac{(2 \Delta x)^3}{3!} u'''_j + \frac{(2 \Delta x)^4}{4!} u^{(4)}_j +\mathcal{O}\left((\Delta x)^5\right) \\
u_{j-1} &= u_j - (\Delta x) u'_j + \frac{(\Delta x)^2}{2} u''_j - \frac{(\Delta x)^3}{3!} u'''_j + \frac{(\Delta x)^4}{4!} u^{(4)}_j +\mathcal{O}\left((\Delta x)^5\right)\\
u_{j} &= u_j \\
u_{j+1} &= u_j + (\Delta x) u'_j + \frac{(\Delta x)^2}{2} u''_j + \frac{(\Delta x)^3}{3!} u'''_j + \frac{(\Delta x)^4}{4!} u^{(4)}_j +\mathcal{O}\left((\Delta x)^5\right)\\
u_{j+2} &= u_j + (2 \Delta x) u'_j + \frac{(2 \Delta x)^2}{2} u''_j + \frac{(2 \Delta x)^3}{3!} u'''_j + \frac{(2 \Delta x)^4}{4!} u^{(4)}_j +\mathcal{O}\left((\Delta x)^5\right)\\
\end{align}

Our goal is to compute coefficients $a_j$ such that $(a_{j-2} u_{j-2} + a_{j-1} u_{j-1}...)/(\Delta x) = u'_j + \mathcal{O}\left((\Delta x)^4\right)$. So we get

\begin{align}
& (a_{j-2} u_{j-2} + a_{j-1} u_{j-1} + a_j u_j + a_{j+1} u_{j+1} +a_{j+2} u_{j+2})/(\Delta x) \\
= & \left( u_j - (2 \Delta x) u'_j + \frac{(2 \Delta x)^2}{2} u''_j -\frac{(2 \Delta x)^3}{3!} u'''_j+\frac{(2 \Delta x)^4}{4!} u^{(4)}_j \right) a_{j-2} \\
& + \left( u_j - (\Delta x) u'_j + \frac{(\Delta x)^2}{2} u''_j - \frac{(\Delta x)^3}{3!} u'''_j+\frac{(\Delta x)^4}{4!} u^{(4)}_j \right) a_{j-1} \\
& + \left( u_j \right) a_{j} \\
& + \left( u_j + (\Delta x) u'_j + \frac{(\Delta x)^2}{2} u''_j + \frac{(\Delta x)^3}{3!} u'''_j+\frac{(\Delta x)^4}{4!} u^{(4)}_j \right) a_{j+1} \\
& + \left( u_j + (2 \Delta x) u'_j + \frac{(2 \Delta x)^2}{2} u''_j + \frac{(2 \Delta x)^3}{3!} u'''_j+\frac{(2 \Delta x)^4}{4!} u^{(4)}_j \right) a_{j+2}
\end{align}

First notice that each time we take a derivative in the Taylor
expansion, we multiply by a $\Delta x$. Notice that this helps to keep
the units consistent (e.g., if $x$ were in units of meters). Let's
just agree to absorb those $\Delta x$'s into the derivatives and next
rearrange terms:

\begin{align}
& a_{j-2} u_{j-2} + a_{j-1} u_{j-1} + a_j u_j + a_{j+1} u_{j+1} + a_{j+2} u_{j+2} \\
= & \left( a_{j-2} + a_{j-1} + a_j + a_{j+1} + a_{j+2} \right) u_j \\
& + \left( -2 a_{j-2} - a_{j-1} + a_{j+1} + 2 a_{j+2} \right) u'_j \\
& + \left( 2^2 a_{j-2} + a_{j-1} + a_{j+1} + 2^2 a_{j+2} \right)/2! u''_j \\
& + \left( -2^3 a_{j-2} - a_{j-1} + a_{j+1} + 2^3 a_{j+2} \right)/3! u'''_j \\
= & u'_j
\end{align}

In order for the above to hold true for any nonzero values of
$\left\{ u_j,u'_j,u''_j,u'''_j,u^{(4)}_j\right\}$, the following set
of equations must also hold:
\begin{align}
0 &= a_{j-2} + a_{j-1} + a_j + a_{j+1} + a_{j+2}\\
1 &= -2 a_{j-2} - a_{j-1} + a_{j+1} + 2 a_{j+2}\\
0 \times 2! &= 2^2 a_{j-2} + a_{j-1} + a_{j+1} + 2^2 a_{j+2}\\
0 \times 3! &= -2^3 a_{j-2} - a_{j-1} + a_{j+1} + 2^3 a_{j+2} \\
0 \times 4! &= 2^4 a_{j-2} + a_{j-1} + a_{j+1} + 2^3 a_{j+2}.
\end{align}

Now we write this expression in matrix form (note that $0!=1$):
\begin{equation}
\left(
\begin{array}{c}
0\times 0! \\
1\times 1! \\
0\times 2! \\
0\times 3! \\
0\times 4! \\
\end{array}
\right)
=
\left(
\begin{array}{ccccc}
 1 &  1 & 1 & 1 & 1 \\
(-2)^1 &(-1)^1 & 0 & 1 & 2 \\
(-2)^2 &(-1)^2 & 0 & 1 & 2^2 \\
(-2)^3 &(-1)^3 & 0 & 1 & 2^3 \\
(-2)^4 &(-1)^4 & 0 & 1 & 2^4 \\
\end{array}
\right)
\left(
\begin{array}{c}
a_{j-2} \\
a_{j-1} \\
a_{j} \\
a_{j+1} \\
a_{j+2} \\
\end{array}
\right)
\end{equation}

So we have reduced the computation of finite difference coefficients
to the inversion of an $N\times N$ matrix equation. Notice that the
elements of the matrix will vary from the one given above if the grid
spacing is not constant, but are otherwise invariant to $\Delta x$.

The inverted matrix reads
\begin{equation}
\left(
\begin{array}{ccccc}
0 & 1/12 & -1/24 & -1/12 & 1/24 \\
0 & -2/3 & 2/3 & 1/6 & -1/6 \\
1 & 0 & -5/4 & 0 & 1/4 \\
0 & 2/3 & 2/3 & -1/6 & -1/6 \\
0 & -1/12 & -1/24 & 1/12 & 1/24 \\
\end{array}
\right)
\label{fourthorder_inv_matrix}
\end{equation}

The coefficients for the $M$th derivative can be immediately read by
multiplying the $(M+1)$st column by $M!$. In short, this matrix gives
the highest-order finite difference derivative coefficients for a stencil size of 5
gridpoints. It can be shown by analyzing cancellations within high
order terms of the Taylor series that the first and
second derivative coefficients are correct to $(\Delta x)^4$ and third
and fourth derivatives are correct to $(\Delta x)^2$.

NRPy+ implements this simple matrix inversion strategy to evaluate finite difference coefficients.